In [9]:
import numpy as np
import pandas as pd
from econml.dml import CausalForestDML
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from preprocessing_ordinal_SMOTE_PSM import propensity_score_matching
from preprocessing_ordinal import apply_ordinal_encoding
from sklearn.model_selection import StratifiedKFold
from scipy import stats
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# =========================
# Globale Font-Settings
# =========================
FONT_SIZE_LABEL = 24
FONT_SIZE_TITLE = 24
FONT_SIZE_LEGEND = 18
FONT_SIZE_TICKS = 24
FONT_SIZE_BAR_LABEL = 24

def get_plotly_font_layout(title_text: str, xaxis_text: str, yaxis_text: str):
    """Erstellt ein zentrales Font-Layout für Plotly-Figuren"""
    return dict(
        title=dict(
            text=title_text,
            font=dict(size=FONT_SIZE_TITLE)
        ),
        xaxis=dict(
            title=dict(text=xaxis_text, font=dict(size=FONT_SIZE_LABEL)),
            tickfont=dict(size=FONT_SIZE_TICKS)
        ),
        yaxis=dict(
            title=dict(text=yaxis_text, font=dict(size=FONT_SIZE_LABEL)),
            tickfont=dict(size=FONT_SIZE_TICKS)
        ),
        template="plotly_white",
        font=dict(size=FONT_SIZE_LABEL),
        legend=dict(
            font=dict(size=FONT_SIZE_LEGEND),
            x=0.7, 
            y=0.95
        ),
        width=800,
        height=650  # Etwas höher wegen Legende unten
    )

def analyze_treatment_balance(df: pd.DataFrame):
    """Analysiert die Balance zwischen Treatment- und Control-Gruppe"""
    treatment_balance = df['Treatment'].mean()
    n_total = len(df)
    n_treated = df['Treatment'].sum()
    n_control = n_total - n_treated
    
    print(f"\nTreatment-Balance:")
    print(f"Gesamt: {n_total} | Treatment: {n_treated} ({treatment_balance:.1%}) | Kontrolle: {n_control}")
    
    return n_treated, n_control

def calculate_smd(df: pd.DataFrame, feature_cols: list) -> pd.DataFrame:
    """Berechnet Standardized Mean Differences für alle Features"""
    smd_results = []
    
    for feature in feature_cols:
        if feature in df.columns and not df[feature].isna().all():
            treated = df[df['Treatment'] == 1][feature]
            control = df[df['Treatment'] == 0][feature]
            
            treated_mean = treated.mean()
            control_mean = control.mean()
            pooled_std = np.sqrt((treated.var() + control.var()) / 2)
            
            if pooled_std > 0:
                smd = (treated_mean - control_mean) / pooled_std
            else:
                smd = 0
            
            smd_results.append({
                'Feature': feature,
                'SMD': smd,
                'Abs_SMD': abs(smd),
                'Imbalanced': abs(smd) > 0.1
            })
    
    return pd.DataFrame(smd_results)


def run_causal_forest_cv(df: pd.DataFrame, feature_cols: list = None, n_splits: int = 5) -> pd.DataFrame:
    """Führt Causal Forest Analyse mit Cross-Validation und econml tune() durch"""
    
    if feature_cols is None:
        feature_cols = [
            "Alter", "2023_Pers_Beurteilung", "2024_Pers_Beurteilung",
            "Betriebszugehoerigkeit", "Gehaltsentwicklung", "2024_Ziele", "2023_Ziele"
        ]
    
    # Daten vorbereiten
    X = df[feature_cols].fillna(df[feature_cols].median()).reset_index(drop=True)
    T = df["Treatment"].astype(int).reset_index(drop=True)
    y = df["Aktiv"].astype(int).reset_index(drop=True)
    
    # Cross-Validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    ITE_all = np.zeros(len(df))
    
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, T)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        T_train, T_test = T.iloc[train_idx], T.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Causal Forest initialisieren
        est = CausalForestDML(
            random_state=42,
            n_estimators=100,
            min_samples_leaf=max(5, min(np.sum(T_train), len(T_train)-np.sum(T_train)) // 10)
        )
        
        try:
            # Hyperparameter tuning (setzt nur die Parameter, trainiert nicht!)
            est.tune(Y=y_train.values, T=T_train.values, X=X_train)
            
            # Beste Hyperparameter ausgeben
            print(f"Fold {fold+1} - Beste Hyperparameter nach Tuning:")
            param_names = [
                'n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf', 
                'max_features', 'min_weight_fraction_leaf', 'max_leaf_nodes',
                'min_impurity_decrease', 'subsample', 'honest', 'inference'
            ]
            
            for param_name in param_names:
                if hasattr(est, param_name):
                    param_value = getattr(est, param_name)
                    print(f"  {param_name}: {param_value}")
            
            # JETZT das Modell mit den getunten Parametern trainieren
            est.fit(Y=y_train.values, T=T_train.values, X=X_train)
            
            # ITE für Testdaten berechnen
            ITE_fold = est.effect(X_test)
            ITE_all[test_idx] = ITE_fold
            
        except Exception as e:
            logger.warning(f"Fold {fold+1} failed: {e}")
            ITE_all[test_idx] = np.nan
    
    # Ergebnisse zusammenstellen
    df_result = df.copy()
    df_result["ITE"] = ITE_all
    
    return df_result

def calculate_uplift_coefficient(df: pd.DataFrame) -> float:
    """Berechnet den Uplift-Koeffizienten (AUC unter der Uplift-Kurve)"""
    uplift_data = create_uplift_curve_data(df)
    
    # Numerische Integration (Trapezregel)
    x = uplift_data['percentage'].values / 100  # Normalisieren auf [0,1]
    y = uplift_data['uplift'].values
    
    uplift_auc = np.trapz(y, x)
    
    return uplift_auc

def calculate_metrics(df_result: pd.DataFrame) -> dict:
    """Berechnet die wichtigsten Metriken"""
    ITE = df_result["ITE"].values
    T = df_result["Treatment"].values
    
    # Statistischer Test gegen 0
    valid_ite = ITE[~np.isnan(ITE)]
    if len(valid_ite) > 0:
        t_stat, p_value = stats.ttest_1samp(valid_ite, 0)
    else:
        t_stat, p_value = np.nan, np.nan
    
    # Gruppengröße
    n_treated = np.sum(T)
    n_control = len(T) - n_treated
    
    # Signifikante ITEs (mit Alpha = 0.05)
    alpha = 0.05
    significant_positive = np.sum(valid_ite > np.percentile(valid_ite, 100 * (1 - alpha/2)))
    significant_negative = np.sum(valid_ite < np.percentile(valid_ite, 100 * (alpha/2)))
    
    # Uplift-Koeffizient
    uplift_coef = calculate_uplift_coefficient(df_result)
    
    return {
        "ATE": np.nanmean(ITE),
        "ATT": np.nanmean(ITE[T == 1]),
        "Median_ITE": np.nanmedian(ITE),
        "Std_ITE": np.nanstd(ITE),
        "t-Test": t_stat,
        "p_Wert": p_value,
        "n_Treatment": n_treated,
        "n_Kontrolle": n_control,
        "Uplift_Koeffizient": uplift_coef,
        "Signifikant_Positive_ITEs": significant_positive,
        "Signifikant_Negative_ITEs": significant_negative
    }

def create_uplift_curve_data(df: pd.DataFrame) -> pd.DataFrame:
    """Erstellt Daten für Uplift-Kurve"""
    df_sorted = df.sort_values('ITE', ascending=False).reset_index(drop=True)
    
    # Berechne kumulative Conversion-Raten für Treatment und Control
    uplift_data = []
    
    for i in range(1, len(df_sorted) + 1):
        # Top i% der Population basierend auf ITE
        top_i = df_sorted.iloc[:i]
        
        # Treatment-Gruppe in Top i%
        treated_in_top = top_i[top_i['Treatment'] == 1]
        control_in_top = top_i[top_i['Treatment'] == 0]
        
        # Conversion-Raten berechnen
        if len(treated_in_top) > 0:
            treatment_conversion = treated_in_top['Aktiv'].mean()
        else:
            treatment_conversion = 0
            
        if len(control_in_top) > 0:
            control_conversion = control_in_top['Aktiv'].mean()
        else:
            control_conversion = 0
        
        # Uplift = Differenz der Conversion-Raten
        uplift = treatment_conversion - control_conversion
        percentage = (i / len(df_sorted)) * 100
        
        uplift_data.append({
            'percentage': percentage,
            'uplift': uplift,
            'treatment_conversion': treatment_conversion,
            'control_conversion': control_conversion,
            'n_treated_top': len(treated_in_top),
            'n_control_top': len(control_in_top)
        })
    
    return pd.DataFrame(uplift_data)

def plot_uplift_curve_single_scenario(scenario_name: str, df_orig: pd.DataFrame, df_psm: pd.DataFrame) -> go.Figure:
    """Erstellt Uplift-Kurve für ein einzelnes Szenario"""
    
    fig = go.Figure()
    
    # Original Daten
    uplift_orig = create_uplift_curve_data(df_orig)
    fig.add_trace(go.Scatter(
        x=uplift_orig['percentage'], 
        y=uplift_orig['uplift'],
        mode='lines',
        name='Originaldaten',
        line=dict(color='#1f77b4', width=3)
    ))
    
    # PSM Daten
    if df_psm is not None:
        uplift_psm = create_uplift_curve_data(df_psm)
        fig.add_trace(go.Scatter(
            x=uplift_psm['percentage'], 
            y=uplift_psm['uplift'],
            mode='lines',
            name='PSM-Daten',
            line=dict(color='#ff7f0e', width=3)
        ))
    
    # Random baseline (Uplift = 0)
    fig.add_trace(go.Scatter(
        x=[0, 100], 
        y=[0, 0],
        mode='lines',
        name='Zufallsbaseline',
        line=dict(color='gray', dash='dot', width=2)
    ))
    
    # Zentrales Layout anwenden
    fig.update_layout(
        **get_plotly_font_layout(
            title_text=f"Uplift-Kurve: {scenario_name}",
            xaxis_text="Prozent der Population (%)",
            yaxis_text="Uplift"
        )
    )
    
    fig.write_image(f"uplift_curve_zeitaufPos_{scenario_name}.pdf")
    return fig

def create_smd_comparison_table(scenario_name: str, smd_orig: pd.DataFrame, smd_psm: pd.DataFrame) -> pd.DataFrame:
    """Erstellt SMD-Vergleichstabelle für ein Szenario"""
    
    # Daten zusammenführen
    merged = smd_orig.merge(smd_psm, on='Feature', suffixes=('_orig', '_psm'))
    
    # Tabelle formatieren (nur die wichtigsten Spalten)
    smd_table = pd.DataFrame({
        'Variable': merged['Feature'],
        'SMD_Original': merged['SMD_orig'].round(4),
        'SMD_PSM': merged['SMD_psm'].round(4),
        'Verbesserung': (merged['Abs_SMD_orig'] - merged['Abs_SMD_psm']).round(4)
    })
    
    # Nach absoluter SMD der Originaldaten sortieren (höchste zuerst)
    merged_sorted = merged.sort_values('Abs_SMD_orig', ascending=False)
    smd_table = pd.DataFrame({
        'Variable': merged_sorted['Feature'],
        'SMD_Original': merged_sorted['SMD_orig'].round(4),
        'SMD_PSM': merged_sorted['SMD_psm'].round(4),
        'Verbesserung': (merged_sorted['Abs_SMD_orig'] - merged_sorted['Abs_SMD_psm']).round(4)
    })
    
    return smd_table

def apply_psm_pipeline(df_scenario: pd.DataFrame) -> pd.DataFrame:
    """Wendet PSM Pipeline an (ohne SMOTE)"""
    try:
        # Direkt PSM anwenden ohne SMOTE
        feature_cols_all = [col for col in df_scenario.columns if col != "Aktiv"]
        feature_cols_psm = [col for col in feature_cols_all if col not in ["Treatment", "Zeit_auf_Position"]]
        
        df_psm = propensity_score_matching(
            df_scenario, 
            treatment_col="Treatment", 
            outcome_col="Aktiv", 
            feature_cols=feature_cols_psm
        )
        
        return df_psm
        
    except Exception as e:
        logger.error(f"Fehler bei PSM Pipeline: {e}")
        return None

def run_single_scenario_analysis(scenario_name: str, df_scenario: pd.DataFrame):
    """Führt vollständige Analyse für ein einzelnes Szenario durch"""
    
    print(f"\n{'='*60}")
    print(f"ANALYSE: {scenario_name}")
    print(f"{'='*60}")
    
    feature_cols = [
        "Alter", "2023_Pers_Beurteilung", "2024_Pers_Beurteilung",
        "Betriebszugehoerigkeit", "Gehaltsentwicklung", "2024_Ziele", "2023_Ziele"
    ]
    
    # === ORIGINALDATEN ===
    print(f"\n--- ORIGINALDATEN ---")
    n_treated_orig, n_control_orig = analyze_treatment_balance(df_scenario)
    
    smd_orig = calculate_smd(df_scenario, feature_cols)
    df_result_orig = run_causal_forest_cv(df_scenario)
    metrics_orig = calculate_metrics(df_result_orig)
    
    print(f"ATE: {metrics_orig['ATE']:.4f}, ATT: {metrics_orig['ATT']:.4f}, p-Wert: {metrics_orig['p_Wert']:.4f}")
    
    # === PSM DATEN ===
    print(f"\n--- PSM-DATEN ---")
    df_psm = apply_psm_pipeline(df_scenario)
    
    if df_psm is not None:
        n_treated_psm, n_control_psm = analyze_treatment_balance(df_psm)
        smd_psm = calculate_smd(df_psm, feature_cols)
        df_result_psm = run_causal_forest_cv(df_psm)
        metrics_psm = calculate_metrics(df_result_psm)
        print(f"ATE: {metrics_psm['ATE']:.4f}, ATT: {metrics_psm['ATT']:.4f}, p-Wert: {metrics_psm['p_Wert']:.4f}")
    else:
        smd_psm = None
        df_result_psm = None
        metrics_psm = None
    
    # === PLOTS UND TABELLEN ERSTELLEN ===
    print(f"\n--- ERSTELLE PLOTS FÜR {scenario_name} ---")
    
    # Uplift-Kurve
    uplift_fig = plot_uplift_curve_single_scenario(scenario_name, df_result_orig, df_result_psm)
    uplift_fig.show()
    
    # SMD Vergleichstabelle
    if smd_psm is not None:
        smd_table = create_smd_comparison_table(scenario_name, smd_orig, smd_psm)
        print(f"\n--- SMD-VERGLEICHSTABELLE: {scenario_name} ---")
        print(smd_table.to_string(index=False))
    
    return {
        'metrics_orig': metrics_orig,
        'metrics_psm': metrics_psm,
        'smd_orig': smd_orig,
        'smd_psm': smd_psm,
        'smd_table': smd_table if smd_psm is not None else None,
        'results_orig': df_result_orig,
        'results_psm': df_result_psm
    }

def run_full_analysis():
    """Hauptanalyse-Funktion"""
    
    # Daten laden
    try:
        df = pd.read_csv("202505_Thesis Daten_anonymisiert.csv", sep=";", encoding="ansi")
    except:
        df = pd.read_csv("202505_Thesis Daten_anonymisiert.csv", sep=";", encoding="utf-8")
    
    df_encoded = apply_ordinal_encoding(df)
    
    # Szenarien definieren
    scenarios = {
        "Hypothese 1": (df_encoded["Zeit_auf_Position"] > 1).astype(int),
        "Hypothese 2": (df_encoded["Zeit_auf_Position"] <= 1).astype(int),
    }
    
    all_results = {}
    summary_orig = []
    summary_psm = []
    
    # Einzelne Szenarien durchführen
    for scenario_name, treatment in scenarios.items():
        df_scenario = df_encoded.copy()
        df_scenario["Treatment"] = treatment
        
        results = run_single_scenario_analysis(scenario_name, df_scenario)
        all_results[scenario_name] = results
        
        # Für Zusammenfassung sammeln
        if results['metrics_orig']:
            summary_orig.append({
                "Szenario": scenario_name,
                **{k: f"{v:.4f}" if isinstance(v, float) else v for k, v in results['metrics_orig'].items()}
            })
        
        if results['metrics_psm']:
            summary_psm.append({
                "Szenario": scenario_name,
                **{k: f"{v:.4f}" if isinstance(v, float) else v for k, v in results['metrics_psm'].items()}
            })
    
    # Zusammenfassungstabellen ausgeben
    print(f"\n{'='*100}")
    print("ZUSAMMENFASSUNG - ORIGINALDATEN")
    print(f"{'='*100}")
    df_summary_orig = pd.DataFrame(summary_orig)
    print(df_summary_orig.to_string(index=False))
    
    print(f"\n{'='*100}")
    print("ZUSAMMENFASSUNG - PSM-DATEN")
    print(f"{'='*100}")
    df_summary_psm = pd.DataFrame(summary_psm)
    print(df_summary_psm.to_string(index=False))
    
    # Zusätzlich: Gesammelte SMD-Tabellen für beide Szenarien
    print(f"\n{'='*100}")
    print("SMD-VERGLEICHSTABELLEN - ÜBERSICHT")
    print(f"{'='*100}")
    for scenario_name, results in all_results.items():
        if results['smd_table'] is not None:
            print(f"\n--- {scenario_name} ---")
            print(results['smd_table'].to_string(index=False))
    
    return df_summary_orig, df_summary_psm, all_results

# Analyse starten
if __name__ == "__main__":
    df_orig, df_psm, all_results = run_full_analysis()


ANALYSE: Hypothese 1

--- ORIGINALDATEN ---

Treatment-Balance:
Gesamt: 9145 | Treatment: 3087 (33.8%) | Kontrolle: 6058


c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was re

Fold 1 - Beste Hyperparameter nach Tuning:
  n_estimators: 100
  max_depth: 3
  min_samples_split: 10
  min_samples_leaf: 247
  max_features: auto
  min_weight_fraction_leaf: 0.0001
  min_impurity_decrease: 0.0
  honest: True
  inference: True


c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was re

Fold 2 - Beste Hyperparameter nach Tuning:
  n_estimators: 100
  max_depth: 3
  min_samples_split: 10
  min_samples_leaf: 247
  max_features: auto
  min_weight_fraction_leaf: 0.0001
  min_impurity_decrease: 0.0
  honest: True
  inference: True


...

Fold 5 - Beste Hyperparameter nach Tuning:
  n_estimators: 100
  max_depth: 3
  min_samples_split: 10
  min_samples_leaf: 484
  max_features: auto
  min_weight_fraction_leaf: 0.0001
  min_impurity_decrease: 0.0
  honest: True
  inference: True


c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\sauer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was re

ATE: 0.0233, ATT: 0.0229, p-Wert: 0.0000

--- ERSTELLE PLOTS FÜR Hypothese 2 ---


INFO:choreographer.browsers.chromium:Chromium init'ed with kwargs {}
INFO:choreographer.browsers.chromium:Found chromium path: C:\Program Files\Google\Chrome\Application\chrome.exe
INFO:choreographer.utils._tmpfile:Temp directory created: C:\Users\sauer\AppData\Local\Temp\tmp8amid3te.
INFO:choreographer.browser_async:Opening browser.
INFO:choreographer.utils._tmpfile:Temp directory created: C:\Users\sauer\AppData\Local\Temp\tmpttoiq_yt.
INFO:choreographer.browsers.chromium:Temporary directory at: C:\Users\sauer\AppData\Local\Temp\tmpttoiq_yt
INFO:kaleido.kaleido:Conforming 1 to file:///C:/Users/sauer/AppData/Local/Temp/tmp8amid3te/index.html
INFO:kaleido.kaleido:Waiting on all navigates
INFO:kaleido.kaleido:All navigates done, putting them all in queue.
INFO:kaleido.kaleido:Getting tab from queue (has 1)
INFO:kaleido.kaleido:Got 0240
INFO:kaleido._kaleido_tab:Processing Uplift_Kurve_Hypothese_2.pdf
INFO:kaleido._kaleido_tab:Sending big command for Uplift_Kurve_Hypothese_2.pdf.
INFO:kal


--- SMD-VERGLEICHSTABELLE: Hypothese 2 ---
              Variable  SMD_Original  SMD_PSM  Verbesserung
Betriebszugehoerigkeit       -0.7623  -0.3227        0.4396
 2023_Pers_Beurteilung       -0.7444   0.0361        0.7083
                 Alter       -0.7032  -0.1584        0.5447
 2024_Pers_Beurteilung       -0.6949   0.0787        0.6162
            2023_Ziele       -0.6419   0.0336        0.6083
    Gehaltsentwicklung       -0.6401   0.0514        0.5887
            2024_Ziele       -0.5991   0.0390        0.5601

ZUSAMMENFASSUNG - ORIGINALDATEN
   Szenario     ATE     ATT Median_ITE Std_ITE    t-Test p_Wert  n_Treatment  n_Kontrolle Uplift_Koeffizient  Signifikant_Positive_ITEs  Signifikant_Negative_ITEs
Hypothese 1 -0.0244 -0.0146    -0.0134  0.0231 -100.8833 0.0000         3087         6058             0.0301                        227                        221
Hypothese 2  0.0244  0.0293     0.0134  0.0231  101.0754 0.0000         6058         3087            -0.0088         